# Identifying Cost Drivers and Improving Financial Sustainability for Hospitals

**Problem statement**

Hospitals operate in a challenging financial landscape, balancing service quality with cost constraints. By analyzing key financial and operational metrics, this project aims to uncover:

- The primary drivers of high costs and low efficiency.
- Differences in financial performance between hospital types (e.g., rural vs. urban).
- Strategies to improve profitability and support financially vulnerable hospitals.


Expected Outcomes:
- Key Cost Drivers Identified: A clear understanding of factors contributing to high costs and inefficiencies.
- Hospital Segmentation: Classification of hospitals into financial health categories to tailor recommendations.
- Actionable Recommendations: Data-driven suggestions for improving cost efficiency, enhancing revenue streams, and supporting financially vulnerable hospitals.


## Imports and utility functions

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def impute_in_beds_range(row, col_name):
    if pd.isnull(row[col_name]):
        n_beds = row['Number of Beds']
        if pd.isnull(n_beds):
            # If 'Number of Beds' is missing, impute with overall median
            median_value = df[col_name].median()
            return median_value
        else:
            # Define the bed range
            min_beds = n_beds - 50
            max_beds = n_beds + 50
            # Filter the dataframe to get hospitals within this bed range and with non-missing salary values
            df_filtered = df[(df['Number of Beds'] >= min_beds) & (df['Number of Beds'] <= max_beds) & (~df[col_name].isnull())]
            # If no hospitals are found in this range, expand the range
            range_expansion = 50 
            while df_filtered.empty:
                min_beds -= range_expansion
                max_beds += range_expansion
                # Ensure min_beds is not less than zero and max_beds doesn't exceed the maximum number of beds
                min_beds = max(min_beds, df['Number of Beds'].min())
                max_beds = min(max_beds, df['Number of Beds'].max())
                # Break the loop if the range cannot be expanded further
                if min_beds == df['Number of Beds'].min() and max_beds == df['Number of Beds'].max():
                    break
                df_filtered = df[(df['Number of Beds'] >= min_beds) & (df['Number of Beds'] <= max_beds) & (~df[col_name].isnull())]
            if df_filtered.empty:
                # If still no hospitals are found, use overall median
                median_value = df[col_name].median()
            else:
                # Compute the median salary variable in this filtered set
                median_value = df_filtered[col_name].median()
            return median_value
    else:
        # If the salary variable is not missing, return the original value
        return row[col_name]

In [3]:
# Load the dataset
file_path = 'Hospital_Provider_Cost_Report_2022.csv'
df = pd.read_csv(file_path)
# Data dictionary: https://data.cms.gov/sites/default/files/2024-03/9756088d-5280-4090-80b9-449d31ef25a3/Cost%20Report%20Data%20Dictionary%20Update.pdf
df.head()

,rpt_rec_num,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,...,Net Income from Service to Patients,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone CHIP,Stand-Alone CHIP Charges
0,717629,344028,STRATEGIC BEHAVIORAL CENTER - GARNER,3200 WATERFIELD ROAD,GARNER,NC,27529-7727,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,717969,180078,PAUL B. HALL REGIONAL MEDICAL CENTER,PAUL B. HALL REGIONAL MEDICAL CENTE,PAINTSVILLE,KY,41240,JOHNSON,99918.0,R,...,634470.0,2151392.0,2785862.0,15.0,2785847.0,0.074617,1114125.0,26708472.0,NaN,NaN
2,719460,520210,MARSHFIELD MEDICAL CENTER-EAU CLAIRE,2116 CRAIG ROAD,EAU CLAIRE,WI,54701,NaN,20740.0,U,...,-4259661.0,845990.0,-3413671.0,151877.0,-3565548.0,0.451411,14333809.0,18797586.0,NaN,NaN
3,719475,63037,COBALT REHAB HOSP WESTMINSTER,6500 WEST 104TH AVE,WESTMINSTER,CO,80020,NaN,79740.0,U,...,-1980346.0,NaN,-1980346.0,NaN,-1980346.0,NaN,NaN,NaN,NaN,NaN
4,721379,340123,AMERICAN HEALTHCARE SYSTEMS,373 NORTH FAYETTEVILLE STREET,ASHEBORO,NC,27204-,RANDOLPH,24660.0,U,...,-5248749.0,-6661015.0,-11909764.0,NaN,-11909764.0,0.160752,1724073.0,6249901.0,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Columns: 117 entries, rpt_rec_num to Stand-Alone CHIP Charges
dtypes: float64(103), int64(4), object(10)
memory usage: 5.4+ MB


In [5]:
df.describe()

,rpt_rec_num,Provider CCN,Medicare CBSA Number,Provider Type,Type of Control,FTE - Employees on Payroll,Number of Interns and Residents (FTE),Total Days Title V,Total Days Title XVIII,Total Days Title XIX,...,Net Income from Service to Patients,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone CHIP,Stand-Alone CHIP Charges
count,6064.000000,6064.000000,5981.000000,6064.000000,6064.000000,5946.000000,1499.000000,134.000000,5913.000000,5051.000000,...,5.979000e+03,5.751000e+03,5.973000e+03,1.778000e+03,5.974000e+03,4600.000000,4.385000e+03,4.373000e+03,9.860000e+02,1.006000e+03
mean,754291.104387,269404.638522,53018.188263,1.708773,4.178595,1174.646171,94.247245,2175.932836,7604.045831,3390.437735,...,-1.213235e+07,2.212219e+07,9.155438e+06,6.840885e+06,7.117901e+06,1.349845,3.172794e+07,1.942532e+08,7.108582e+05,4.443109e+06
std,10770.303579,160322.536510,34012.617263,1.431237,3.200321,13569.179424,170.730012,4148.544626,12124.613554,8052.593393,...,1.469759e+08,9.444758e+07,1.285488e+08,4.677254e+07,1.289053e+08,53.870625,8.750084e+07,4.248297e+08,3.114268e+06,2.170038e+07
min,717629.000000,10001.000000,10180.000000,1.000000,1.000000,0.050000,0.010000,1.000000,1.000000,1.000000,...,-1.625376e+09,-2.963724e+08,-1.224029e+09,-5.686516e+08,-1.224029e+09,0.041931,-1.449717e+07,1.000000e+01,5.400000e+01,7.200000e+01
25%,745140.750000,140157.250000,26420.000000,1.000000,2.000000,129.112500,8.395000,110.000000,989.000000,121.000000,...,-1.248723e+07,6.433410e+05,-3.266985e+06,1.000000e+00,-3.868518e+06,0.197872,2.851322e+06,1.118957e+07,1.748900e+04,9.999850e+04
50%,755650.500000,260021.000000,39300.000000,1.000000,2.000000,288.250000,28.330000,722.000000,2819.000000,680.000000,...,-2.037805e+06,2.970702e+06,1.004401e+06,9.739300e+04,7.509730e+05,0.289937,9.163034e+06,5.166032e+07,7.285500e+04,4.802065e+05
75%,762052.750000,393035.250000,99916.000000,1.000000,5.000000,902.735000,91.025000,2627.250000,9297.000000,3083.000000,...,2.852722e+06,1.060829e+07,8.285527e+06,1.888425e+06,7.395911e+06,0.475700,2.784502e+07,1.940467e+08,2.834080e+05,2.083000e+06
max,776832.000000,713029.000000,99966.000000,10.000000,13.000000,987647.000000,1734.240000,32548.000000,176816.000000,175703.000000,...,7.864650e+09,1.794124e+09,7.903916e+09,7.197225e+08,7.919570e+09,3636.891704,3.173354e+09,7.968634e+09,4.481714e+07,3.791126e+08


## Data Wrangling

### Handle missing values

In [6]:
missing_values = df.isnull().sum()

In [7]:
missing_percentage = (missing_values / len(df)) * 100
missing_percentage.sort_values(ascending=False)

Outlier Payments For Discharges                        100.000000
DRG Amounts Other Than Outlier Payments                100.000000
Hospital Total Days Title V For Adults & Peds           97.955145
Total Discharges Title V                                97.922164
Hospital Total Discharges Title V For Adults & Peds     97.922164
                                                          ...    
Provider Type                                            0.000000
Type of Control                                          0.000000
Fiscal Year Begin Date                                   0.000000
Provider CCN                                             0.000000
rpt_rec_num                                              0.000000
Length: 117, dtype: float64

In [8]:
# Drop variables with high missingness or irrelavant
drop_vars = [
    'Outlier Payments For Discharges',
    'DRG Amounts Other Than Outlier Payments',
    'Hospital Total Days Title V For Adults & Peds',
    'Hospital Total Discharges Title V For Adults & Peds',
    'Total Discharges Title V',
    'Total Days Title V',
    'Wage-Related Costs (RHC/FQHC)',
    'Total Other Expenses',
    'County',
    'Street Address']

df.drop(columns=drop_vars, inplace=True)

In [9]:
# List of variables to fill with zero - Missingness likely indicates the absence of a feature or activity
fill_zero_vars = [
    # Financial variables
    'Notes Receivable',
    'Unsecured Loans',
    'Mortgage Payable',
    'Notes Payable',
    'Investments',
    'Temporary Investments',
    'Deferred Income',
    'Payroll Taxes Payable',
    'Notes and Loans Payable (Short Term)',
    'Other Current Assets',
    'Other Long Term Liabilities',
    'Other Assets',
    'Total Other Assets',
    'Less: Allowances for Uncollectible Notes and Accounts Receivable',
    'Other Current Liabilities',
    'Total Long Term Liabilities',
    # Operational variables
    'Number of Interns and Residents (FTE)',
    'Wage Related Costs for Interns and Residents',
    'Wage Related Costs for Part - A Teaching Physicians',
    'Contract Labor: Direct Patient Care',
    # Program-specific variables
    'Net Revenue from Stand-Alone CHIP',
    'Stand-Alone CHIP Charges',
    'Managed Care Simulated Payments',
    'Total IME Payment',
    'DRG Amounts Before October 1',
    'DRG Amounts After October 1',
    'Disproportionate Share Adjustment',
    'Allowable DSH Percentage',
    # Asset variables
    'Health Information Technology Designated Assets',
    'Leasehold Improvements',
    'Minor Equipment Depreciable',
    'Fixed Equipment',
    'Land Improvements',
    'Land',
    'Buildings',
    'Major Movable Equipment',
    # Charity care and bad debt
    'Cost of Charity Care',
    'Total Bad Debt Expense',
    'Cost of Uncompensated Care',
    'Total Unreimbursed and Uncompensated Care',
    # Medicaid variables
    'Medicaid Charges',
    'Net Revenue from Medicaid',
    # Title variables
    'Hospital Total Discharges Title XIX For Adults & Peds',
    'Total Discharges Title XIX',
    'Hospital Total Days Title XIX For Adults & Peds',
    'Total Days Title XIX',
    'Hospital Total Days Title XVIII For Adults & Peds',
    'Total Days Title XVIII',
    'Hospital Total Discharges Title XVIII For Adults & Peds',
    'Total Discharges Title XVIII',
    # Outpatient variables
    'Outpatient Revenue',
    'Outpatient Total Charges',
    # Other financial variables
    'Prepaid Expenses',
    'Inventory',
    'Salaries, Wages, and Fees Payable'
]

In [10]:
for col in fill_zero_vars:
    df[col] = df[col].fillna(0)        

In [11]:
# Impute Values Based on Number of Beds Range - value has correlation with beds
vars_corr_beds = [
    'Wage-Related Costs (Core)', 
    'Total Salaries (adjusted)', 
    'Inpatient Revenue', 
    'Total Patient Revenue', 
    'Net Patient Revenue', 
    'Total Fixed Assets',
    'Total Current Liabilities',
    'Total Liabilities',
    'Total Current Assets',
    'Total Other Income',
    'General Fund Balance',
    'Total Fund Balances',
    'Total Liabilities and Fund Balances',
    'Total Assets',
    'Depreciation Cost',
    "Less Contractual Allowance and Discounts on Patients' Accounts",
]
for col in vars_corr_beds:
    df[col] = df.apply(lambda row: impute_in_beds_range(row, col), axis=1)    

In [12]:
# Compute 'Cost To Charge Ratio' Where Missing

missing_ratio = df['Cost To Charge Ratio'].isnull()

# Ensure 'Total Costs' and 'Combined Outpatient + Inpatient Total Charges' are not zero to avoid division by zero
valid_indices = (df['Total Costs'] != 0) & (df['Combined Outpatient + Inpatient Total Charges'] != 0)
indices_to_compute = missing_ratio & valid_indices
df.loc[indices_to_compute, 'Cost To Charge Ratio'] = (
    df.loc[indices_to_compute, 'Total Costs'] /
    df.loc[indices_to_compute, 'Combined Outpatient + Inpatient Total Charges']
)

In [13]:
df['Cost To Charge Ratio'].isnull().sum()

83

In [14]:
# Remove if no 'Total Costs' and 'Combined Outpatient + Inpatient Total Charges'
df = df[~df['Cost To Charge Ratio'].isnull()]
df['Cost To Charge Ratio'].isnull().sum()

0

In [15]:
# Re-check missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

print(missing_percentage[missing_percentage > 0].sort_values(ascending=False).to_string())

Cash on Hand and in Banks                                                  7.824778
Accounts Payable                                                           5.366995
Accounts Receivable                                                        4.898846
FTE - Employees on Payroll                                                 0.585186
Hospital Total Days (V + XVIII + XIX + Unknown) For Adults & Peds          0.351112
Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults & Peds    0.317673
Total Discharges (V + XVIII + XIX + Unknown)                               0.317673
Total Days (V + XVIII + XIX + Unknown)                                     0.300953
Hospital Number of Beds For Adults & Peds                                  0.250794
Hospital Total Bed Days Available For Adults & Peds                        0.234075
Total Bed Days Available                                                   0.183916
Number of Beds                                                             0

In [16]:
financial_vars = ['Cash on Hand and in Banks', 'Accounts Payable', 'Accounts Receivable']

for col in financial_vars:
    if col in df.columns:
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)

In [17]:
# Re-check missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

print(missing_percentage[missing_percentage > 0].sort_values(ascending=False).to_string())

FTE - Employees on Payroll                                                 0.585186
Hospital Total Days (V + XVIII + XIX + Unknown) For Adults & Peds          0.351112
Total Discharges (V + XVIII + XIX + Unknown)                               0.317673
Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults & Peds    0.317673
Total Days (V + XVIII + XIX + Unknown)                                     0.300953
Hospital Number of Beds For Adults & Peds                                  0.250794
Hospital Total Bed Days Available For Adults & Peds                        0.234075
Number of Beds                                                             0.183916
Total Bed Days Available                                                   0.183916
Total Income                                                               0.133757
Net Income                                                                 0.117037
Number of Beds + Total for all Subproviders                                0

In [18]:
df.isnull().any(axis=1).sum()

65

In [19]:
df = df.dropna()
df.isnull().sum().any()

False

### Correct Data Type

## EDA

## Data Preprocessing

## Modeling

## Others